In [1]:
import cv2
import datetime
import numpy as np
import keras_tuner as kt
import os
import pandas as pd
import pickle
import random
import tensorflow as tf

from itertools import product
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
from random import choice, sample
from tensorflow import keras
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Input, Flatten, Dense, GlobalMaxPool2D, GlobalAvgPool2D, Conv1D, MaxPool1D, Concatenate, Add, Lambda, Multiply, Dropout, Subtract
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image as image_p
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.constraints import max_norm
from scipy import stats

import time

In [2]:
existing_train_data = pd.read_csv('/home/roman/Work/kinship/train_ds.csv')
new_train_data = pd.read_csv('/home/roman/Work/kinship/train2.csv')
new_new_train_data = pd.read_csv('/home/roman/Work/kinship/train3.csv')

# Augment dataset by generating all possible combos of known kin

Assumes images in same folder = same person

In [3]:
relationship_set = set()
non_relationship_set = set()
relationship_set_2 = set()

folder_to_images_map = {}
all_people_folders = set()
base_path = '/home/roman/Work/kinship/train/train-faces/'
for index, row in existing_train_data.iterrows():
    if row['relationship'] == 1.0:
        p1_path = row['p1'].split('/')
        p2_path = row['p2'].split('/')
        p1_rel_path = p1_path[0] + '/' + p1_path[1]
        p2_rel_path = p2_path[0] + '/' + p2_path[1]
        
        complete_p1_path = os.path.join(base_path, p1_rel_path)
        complete_p2_path = os.path.join(base_path, p2_rel_path)
        
        # Maintain sets for people-folders and known positive relationships
        
        relationship_set.add((complete_p1_path, complete_p2_path))
        relationship_set.add((complete_p2_path, complete_p1_path))
        relationship_set_2.add((p1_path[0], p2_path[0]))
        relationship_set_2.add((p2_path[0], p1_path[0]))
        all_people_folders.add(complete_p1_path)
        all_people_folders.add(complete_p2_path)
        
        # Create mapping between folder names and their image files that ar econtained
        folder_to_images_map[complete_p1_path] = set()
        folder_to_images_map[complete_p2_path] = set()
        
        for root, dirs, files in os.walk(complete_p1_path, topdown=False):
            for name in files:
                folder_to_images_map[complete_p1_path].add(os.path.join(complete_p1_path, name))
        
        for root, dirs, files in os.walk(complete_p2_path, topdown=False):
            for name in files:
                folder_to_images_map[complete_p2_path].add(os.path.join(complete_p2_path, name))
        
for index, row in new_train_data.iterrows():
    if row['relationship'] == 1:
        p1_path = row['p1'].split('/')
        p2_path = row['p2'].split('/')
        p1_rel_path = p1_path[0] + '/' + p1_path[1]
        p2_rel_path = p2_path[0] + '/' + p2_path[1]
        
        complete_p1_path = os.path.join(base_path, p1_rel_path)
        complete_p2_path = os.path.join(base_path, p2_rel_path)
        
        relationship_set.add((complete_p1_path, complete_p2_path))
        relationship_set.add((complete_p2_path, complete_p1_path))
        relationship_set_2.add((p1_path[0], p2_path[0]))
        relationship_set_2.add((p2_path[0], p1_path[0]))
        all_people_folders.add(complete_p1_path)
        all_people_folders.add(complete_p2_path)
        
        folder_to_images_map[complete_p1_path] = set()
        folder_to_images_map[complete_p2_path] = set()
        
        for root, dirs, files in os.walk(complete_p1_path, topdown=False):
            for name in files:
                folder_to_images_map[complete_p1_path].add(os.path.join(complete_p1_path, name))
        
        for root, dirs, files in os.walk(complete_p2_path, topdown=False):
            for name in files:
                folder_to_images_map[complete_p2_path].add(os.path.join(complete_p2_path, name))

for index, row in new_new_train_data.iterrows():
    # print(new_new_train_data)
    if row['relationship'] == 1:
        p1_path = row['p1'].split('/')
        p2_path = row['p2'].split('/')
        p1_rel_path = p1_path[0] + '/' + p1_path[1]
        p2_rel_path = p2_path[0] + '/' + p2_path[1]
        
        complete_p1_path = os.path.join(base_path, p1_rel_path)
        complete_p2_path = os.path.join(base_path, p2_rel_path)
        
        relationship_set.add((complete_p1_path, complete_p2_path))
        relationship_set.add((complete_p2_path, complete_p1_path))
        relationship_set_2.add((p1_path[0], p2_path[0]))
        relationship_set_2.add((p2_path[0], p1_path[0]))
        all_people_folders.add(complete_p1_path)
        all_people_folders.add(complete_p2_path)
        
        folder_to_images_map[complete_p1_path] = set()
        folder_to_images_map[complete_p2_path] = set()
        
        for root, dirs, files in os.walk(complete_p1_path, topdown=False):
            for name in files:
                folder_to_images_map[complete_p1_path].add(os.path.join(complete_p1_path, name))
        
        for root, dirs, files in os.walk(complete_p2_path, topdown=False):
            for name in files:
                folder_to_images_map[complete_p2_path].add(os.path.join(complete_p2_path, name))
                
# create all possible combos of pairings
for a, b in product(list(all_people_folders), list(all_people_folders)):
    candidate_pair = (a,b)
    a_person_id = a.split('/')[7]
    b_person_id = b.split('/')[7]
    if candidate_pair not in relationship_set and a_person_id!=b_person_id:
        non_relationship_set.add(candidate_pair)
        
for a, b in product(list(all_people_folders), list(all_people_folders)):
    candidate_pair = (a,b)
    a_person_id = a.split('/')[7]
    b_person_id = b.split('/')[7]
    person_candidate_pair = (a_person_id, b_person_id)
    if candidate_pair not in non_relationship_set and a_person_id!=b_person_id and person_candidate_pair in relationship_set_2:
        relationship_set.add(candidate_pair)
    

# Train/Val generators

Perturbs half the images and leaves the rest alone. Color transfer non-functional, so we're only using affine image transormations of rotation, resize, and translation.

In [4]:
def read_img(path1, path2, perturb=False, color_transfer_bool=False):
    img1 = image_p.load_img(path1, target_size=(224, 224))
    img1 = np.array(img1).astype(np.float)
    img2 = image_p.load_img(path2, target_size=(224, 224))
    img2 = np.array(img2).astype(np.float)
    return preprocess_input(img1, version=2), preprocess_input(img2, version=2)

def gen(relationship_tuples, non_relationship_tuples, person_to_images_map, batch_size=16):
    while True:
        
        # color transfer and perturb
        relationship_batch_tuples = sample(relationship_tuples, batch_size//2)
        non_relationship_batch_tuples = sample(non_relationship_tuples, batch_size//2)
        
        incomplete_path_batch_tuples = relationship_batch_tuples + non_relationship_batch_tuples
        
        batch_tuples = [(sample(person_to_images_map[p1], 1)[0], sample(person_to_images_map[p2], 1)[0]) for p1, p2 in incomplete_path_batch_tuples]
            
        labels = batch_size//2*[1] + batch_size//2*[0]
        
        # print(batch_tuples)
        # print(labels)
        
        processed_batch_image_tuples = [read_img(img1_path, img2_path, perturb=False, color_transfer_bool=False) for img1_path, img2_path in batch_tuples]
        unzipped_processed_batch_images = list(zip(*processed_batch_image_tuples))
        
        X1 = np.array(list(unzipped_processed_batch_images[0]))

        X2 = np.array(list(unzipped_processed_batch_images[1]))

        yield [X1, X2], np.array(labels)
        
        

In [5]:
train_gen = gen(relationship_set, non_relationship_set, folder_to_images_map)

# Create relationship/nonrelationship data based on set differencing and one person per folder assumption

In [6]:

train_file_path = "/home/roman/Work/kinship/train_ds.csv"
train_folders_path = "/home/roman/Work/kinship/train/train-faces/"

def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

def get_random_validation_sets(all_relationships, all_non_relationships, n):

    relationship_set =  [] 
    for validation_set in split(list(all_relationships), n):
        relationship_set.append(set(validation_set))
    
    non_relationship_set =  [] 
    for validation_set in split(list(all_non_relationships), n):
        non_relationship_set.append(set(validation_set))
    
    return relationship_set, non_relationship_set

def get_random_train_val_sets(all_relationships, all_non_relationships, n, person_to_images_map):
    generators = []
    
    relationship_val_sets, non_relationship_val_sets = get_random_validation_sets(all_relationships, all_non_relationships, n)
    
    for val_relationship_set, val_non_relationship_set in zip(relationship_val_sets, non_relationship_val_sets):
        train_relationship_set = all_relationships.difference(val_relationship_set)
        train_non_relationship_set = all_non_relationships.difference(val_non_relationship_set)
        yield gen(train_relationship_set, train_non_relationship_set, person_to_images_map), gen(val_relationship_set, val_non_relationship_set, person_to_images_map)
 

In [7]:
k_folds = 5

In [8]:
train_val_gen = get_random_train_val_sets(relationship_set, non_relationship_set, k_folds, folder_to_images_map)

In [9]:
relationship_val_sets, non_relationship_val_sets = get_random_validation_sets(relationship_set, non_relationship_set, k_folds)

## Save validation sets

In [10]:
with open(f"validation_sets_perturbed_with_test_{datetime.date.today().strftime('%Y_%m_%d')}.pickle", 'wb') as f:
    pickle.dump((relationship_val_sets, non_relationship_val_sets), f)

# Callbacks

In [11]:
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('val_acc') >= .8:
            self.model.stop_training = True

In [12]:
reduce_on_plateau =  ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.3, patience=2, verbose=1)

max_val_acc = CustomCallback()

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=12, min_delta=.0075)

log_dir = '/home/roman/Work/kinship/logs'

# tensorboard_callback = TensorBoard(log_dir=log_dir)

callbacks_list = [
    # max_val_acc, 
    reduce_on_plateau, 
    stop_early
]

# Model Initialization

In [13]:
# Reference architecture: https://arxiv.org/abs/2006.00143v2
def baseline_model():
    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))

    base_model = VGGFace(model='resnet50', include_top=False)

    for x in base_model.layers[:-3]:
        x.trainable = True

    x1 = base_model(input_1)
    x2 = base_model(input_2)


    x1=GlobalMaxPool2D()(x1)
    x2=GlobalAvgPool2D()(x2)
    
    x3 = Subtract()([x1, x2])
    x3 = Multiply()([x3, x3])
    
    x1_ = Multiply()([x1, x1])
    x2_ = Multiply()([x2, x2])
    x4 = Subtract()([x1_, x2_])
    
    x5 = Multiply()([x1, x2])
    
    x = Concatenate(axis=-1)([x3, x4, x5])
#     x = Dense(512, activation="relu")(x)
#     x = Dropout(0.03)(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.2)(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.2)(x)
    out = Dense(1, activation="sigmoid")(x)


    model = Model([input_1, input_2], out)

    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))
#     model.compile(loss=[focal_loss(alpha=.25, gamma=2)], metrics=['acc'], optimizer=Adam(0.00003))
#     model.compile(loss=[focal_loss(alpha=.25, gamma=2)], metrics=['acc'], optimizer=Adam(0.00001))
    model.summary()

    return model

In [14]:
model = baseline_model()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vggface_resnet50 (Functional)   (None, None, None, 2 23561152    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
global_max_pooling2d (GlobalMax (None, 2048)         0           vggface_resnet50[0][0]       

# Model training

In [15]:
need_to_train = [1]
train_all = True
for i, (train_gen, val_gen) in enumerate(train_val_gen):
    if i in need_to_train or train_all:
        model = baseline_model()
        file_path = f"pwc_perturbed_more_samples_tes_data_random_validation_ensemble_v3_{i}.h5"
        history = model.fit(
            train_gen, 
            use_multiprocessing=False,
            validation_data=val_gen, 
            epochs=16, callbacks=callbacks_list, 
            steps_per_epoch=32, validation_steps=128
        )
        model.save(file_path)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vggface_resnet50 (Functional)   (None, None, None, 2 23561152    input_4[0][0]                    
                                                                 input_5[0][0]                    
__________________________________________________________________________________________________
global_max_pooling2d_1 (GlobalM (None, 2048)         0           vggface_resnet50[0][0]     

<ipython-input-4-e024f4f20b08>:12: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  relationship_batch_tuples = sample(relationship_tuples, batch_size//2)
<ipython-input-4-e024f4f20b08>:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_relationship_batch_tuples = sample(non_relationship_tuples, batch_size//2)
<ipython-input-4-e024f4f20b08>:17: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  batch_tuples = [(sample(person_to_images_map[p1], 1)[0], sample(person_to_images_map[p2], 1)[0]) for p1, p2 in incomplete_path_batch_tuples]
<ipython-input-4-e024f4f20b08>:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar ty

Epoch 1/16
32/32 [==============================] - 59s 2s/step - loss: 5.6584 - acc: 0.5176 - val_loss: 5.2194 - val_acc: 0.5010
Epoch 2/16
32/32 [==============================] - 46s 1s/step - loss: 3.7803 - acc: 0.4997 - val_loss: 4.3201 - val_acc: 0.4917
Epoch 3/16
32/32 [==============================] - ETA: 0s - loss: 3.5061 - acc: 0.5020

KeyboardInterrupt: 

# Export predictions for a single model inside ensemble

Make sure class predictions match test set proportions

In [ ]:
def read_img_for_prediction(path):
    img = image_p.load_img(path, target_size=(224, 224))
    img = np.array(img).astype(np.float)
    return preprocess_input(img, version=2)

In [ ]:
import datetime 
submission = pd.read_csv('/home/roman/Work/kinship/test_ds2.csv')
prediction_list = []
test_path = "/home/roman/Work/kinship/test2/"
for j in range(k_folds):
    print(f'exporting prediction number {j}')
    file_path = f"pwc_perturbed_more_samples_tes_data_random_validation_ensemble_v3_{j}.h5"
    model = load_model(file_path)
    
    predictions = []
    
    for i in range(0, len(submission.p1.values), 32):

        X1 = submission.p1.values[i:i+32]
        X1 = np.array([read_img_for_prediction(test_path + x) for x in X1])

        X2 = submission.p2.values[i:i+32]
        X2 = np.array([read_img_for_prediction(test_path + x) for x in X2])

        pred = model.predict([X1, X2]).ravel().tolist()
        predictions += pred
        
    np.array(predictions)
        
    prediction_list.append(predictions)
    
    d = {'index': np.arange(0, len(predictions), 1), 'label': predictions}
    submissionfile = pd.DataFrame(data=d)
    # Balance prediction class ratio with expected test class ratio
    score_for_0 = submissionfile['label'].quantile(0.493333)
    preds1 = submissionfile['label'].apply(lambda s: 1 if s>score_for_0 else 0).to_frame('label').reset_index()['label']
    submissionfile['label'] = preds1
    submissionfile['label'] = submissionfile['label'].astype(int)
    submissionfile.to_csv(f"/home/roman/Work/kinship/pwc_perturbed_more_samples_random_validation_ensemble_v7_{datetime.date.today().strftime('%Y_%m_%d')}_{j}.csv", index=False)

    del model


In [ ]:
import glob
import datetime 
submission = pd.read_csv('/home/roman/Work/kinship/test_ds2.csv')
# prediction_list = []
test_path = "/home/roman/Work/kinship/test2/"
models = glob.glob('./*.h5')
for index, model in enumerate(models):
    if index > 3:
        print(model)
        model = load_model(model)

        predictions = []
        batch_size = 256
        for i in range(0, len(submission.p1.values), batch_size):

            X1 = submission.p1.values[i:i+batch_size]
            X1 = np.array([read_img_for_prediction(test_path + x) for x in X1])

            X2 = submission.p2.values[i:i+batch_size]
            X2 = np.array([read_img_for_prediction(test_path + x) for x in X2])

            pred = model.predict([X1, X2]).ravel().tolist()
            predictions += pred

        np.array(predictions)

        prediction_list.append(predictions)

        d = {'index': np.arange(0, len(predictions), 1), 'label': predictions}
        submissionfile = pd.DataFrame(data=d)
        # Balance prediction class ratio with expected test class ratio
        score_for_0 = submissionfile['label'].quantile(0.493333)
        preds1 = submissionfile['label'].apply(lambda s: 1 if s>score_for_0 else 0).to_frame('label').reset_index()['label']
        submissionfile['label'] = preds1
        submissionfile['label'] = submissionfile['label'].astype(int)
        submissionfile.to_csv(f"/home/roman/Work/kinship/pwc_perturbed_more_samples_random_validation_ensemble_v7_{datetime.date.today().strftime('%Y_%m_%d')}_{index}.csv", index=False)

        del model


# Ensemble predictions

In [ ]:
func = lambda x: int(round(x,0))
final_predictions = [list(map(func, i)) for i in prediction_list]

final_predictions = stats.mode(final_predictions)[0][0]
d = {'index': np.arange(0, len(p), 1), 'label': final_predictions}
submissionfile = pd.DataFrame(data=d)
submissionfile['label'] = submissionfile['label'].astype(int)
submissionfile.to_csv(f"/home/roman/Work/kinship/giant_ensemble_v2_{datetime.date.today().strftime('%Y_%m_%d')}.csv", index=False)